# Universidad del Valle de Guatemala
# (CC3093) Security Data Science
# Proyecto 2 - Modelo de Detección de Fraude
# Santiago Taracena Puga (20017)

## Documento: Desarrollo del modelo 2 (Random Forest)

## Introducción

En el ámbito de la seguridad informática y la detección de fraudes, el análisis de datos se ha convertido en un pilar fundamental para identificar patrones y anomalías que puedan indicar actividades fraudulentas. En este contexto, el uso de modelos de Machine Learning y Deep Learning ha demostrado ser una herramienta poderosa para detectar transacciones fraudulentas en tiempo real, permitiendo a las instituciones financieras proteger los activos de sus clientes y salvaguardar la integridad de sus sistemas.

El presente proyecto surge en colaboración entre el Convenio PLUS TI y la Universidad del Valle, con el objetivo de investigar la viabilidad y eficacia del entrenamiento incremental en modelos de aprendizaje automático y profundo aplicados a la detección de fraudes en transacciones de tarjeta de crédito. Este estudio se realiza utilizando un dataset simulado que abarca transacciones realizadas desde el 1 de enero de 2019 hasta el 31 de diciembre de 2020, donde se incluyen tanto transacciones legítimas como fraudulentas.

El enfoque adoptado se centra en dos partes principales. En primer lugar, se lleva a cabo una investigación teórica exhaustiva sobre el entrenamiento incremental en los algoritmos seleccionados, destacando sus capacidades, limitaciones y las mejores prácticas asociadas. Esta revisión bibliográfica proporcionará un marco conceptual sólido para comprender el contexto y los desafíos del entrenamiento incremental en el ámbito de la detección de fraudes.

En segunda instancia, se procede con la implementación práctica de modelos de Machine Learning y Deep Learning, utilizando algoritmos como Redes Neuronales Artificiales (ANN), LightGBM, XGBoost, Random Forest y Máquinas de Vectores de Soporte (SVM). Se parte de un análisis exploratorio de datos (EDA) para comprender las características del dataset, seguido de un proceso de feature engineering para crear nuevas características relevantes para la detección de fraudes. Además, se exploran técnicas de manejo de datos desequilibrados, afinación de hiperparámetros y estrategias de early stopping para mejorar el rendimiento de los modelos.

La evaluación de los modelos se realiza comparando su rendimiento antes y después del entrenamiento incremental, utilizando métricas como ROC-AUC, precisión, recall y F1-score. Se presta especial atención a la capacidad de detección de transacciones fraudulentas y a cualquier pérdida significativa de rendimiento que pueda surgir como resultado del entrenamiento incremental.

Finalmente, se propone una metodología para decidir cuándo es preferible realizar un reentrenamiento total en lugar de uno incremental, basándose en la literatura existente y en los resultados experimentales obtenidos. Esta metodología se valida empíricamente utilizando los modelos entrenados, y se proporcionan conclusiones y recomendaciones para futuras investigaciones o aplicaciones prácticas en el campo de la seguridad de datos financieros.

## Parte 1: Entrenamiento Incremental

### Desarrollo del modelo 2 (Random Forest)

In [25]:
# Instrucción para importar la librería pandas
import pandas as pd

In [26]:
# Lectura del dataset con los features necesarios
data = pd.read_csv("./data/dataset-v2.csv")
data.head()

,is_fraud,transaction_year,transaction_month,transaction_day,transaction_hour,transaction_minute,transaction_second,transaction_cc_num_length,transaction_cc_num_unique_digits_count,transaction_cc_num_digits_sum,...,transaction_amount_change,transaction_lat_change,transaction_long_change,location_change,transaction_count_per_category,avg_amount_per_category,transaction_amount_last_7_days,transaction_amount_last_month,std_transaction_amount,cv_transaction_amount
0,0,2019,1,1,0,6,56,15,8,54,...,NaN,NaN,NaN,0.000000,5654,132.379484,71.22,71.22,126.447035,1.673431
1,0,2019,1,1,0,17,16,15,7,54,...,-24.94,12.8254,8.3093,1556.500349,2112,59.030284,117.50,117.50,126.447035,1.673431
2,0,2019,1,1,0,22,36,15,7,76,...,-42.62,3.1423,-48.0660,3714.630408,4419,94.525207,121.16,121.16,126.447035,1.673431
3,0,2019,1,1,0,32,15,15,8,72,...,126.74,-0.8190,34.0860,2775.387784,5654,132.379484,251.56,251.56,126.447035,1.673431
4,0,2019,1,1,1,9,57,15,7,54,...,-126.61,-2.3233,13.9800,1226.358716,2541,90.435702,255.35,255.35,126.447035,1.673431


In [27]:
# Instrucción para importar LabelEncoder y codificar variables categóricas
from sklearn.preprocessing import LabelEncoder

In [28]:
# Instancia y retorno de LabelEncoder
encoder = LabelEncoder()
encoder

LabelEncoder()

In [29]:
# Lista de variables categóricas a codificar
labels_to_encode = [
    "transaction_merchant",
    "transaction_category",
    "transaction_gender",
    "transaction_job",
    "transaction_time_of_day",
    "transaction_time_period",
    "transaction_season",
]
labels_to_encode

['transaction_merchant',
 'transaction_category',
 'transaction_gender',
 'transaction_job',
 'transaction_time_of_day',
 'transaction_time_period',
 'transaction_season']

In [30]:
# Ciclo que codifica todas las variables categóricas listadas
for label in labels_to_encode:
    data[label] = encoder.fit_transform(data[label])

# Primeras cinco filas del dataset para observar la codificación
data.head()

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

,is_fraud,transaction_year,transaction_month,transaction_day,transaction_hour,transaction_minute,transaction_second,transaction_cc_num_length,transaction_cc_num_unique_digits_count,transaction_cc_num_digits_sum,...,transaction_amount_change,transaction_lat_change,transaction_long_change,location_change,transaction_count_per_category,avg_amount_per_category,transaction_amount_last_7_days,transaction_amount_last_month,std_transaction_amount,cv_transaction_amount
0,0,2019,1,1,0,6,56,15,8,54,...,NaN,NaN,NaN,0.000000,5654,132.379484,71.22,71.22,126.447035,1.673431
1,0,2019,1,1,0,17,16,15,7,54,...,-24.94,12.8254,8.3093,1556.500349,2112,59.030284,117.50,117.50,126.447035,1.673431
2,0,2019,1,1,0,22,36,15,7,76,...,-42.62,3.1423,-48.0660,3714.630408,4419,94.525207,121.16,121.16,126.447035,1.673431
3,0,2019,1,1,0,32,15,15,8,72,...,126.74,-0.8190,34.0860,2775.387784,5654,132.379484,251.56,251.56,126.447035,1.673431
4,0,2019,1,1,1,9,57,15,7,54,...,-126.61,-2.3233,13.9800,1226.358716,2541,90.435702,255.35,255.35,126.447035,1.673431


In [31]:
# Separar características y etiquetas
X = data.drop("is_fraud", axis=1).fillna(0)
y = data["is_fraud"]

#### 1.2.3. Manejo de Datos Desequilibrados

El manejo de datos desequilibrados es una parte crucial en la construcción de modelos de detección de fraude, dado que la proporción de transacciones fraudulentas es extremadamente baja en comparación con las transacciones legítimas. En nuestro caso, el dataset presentaba un desequilibrio significativo, con el 99.5% de las transacciones etiquetadas como no fraudulentas y solo el 0.5% como fraudulentas. Este desequilibrio puede llevar a que los modelos de machine learning se inclinen a predecir siempre la clase mayoritaria, resultando en una alta exactitud superficial pero una baja efectividad en la detección de fraude.

Para abordar este problema, se implementaron dos técnicas principales: RandomUnderSampler y SMOTE (Synthetic Minority Over-sampling Technique). Estas técnicas fueron seleccionadas para equilibrar las clases y mejorar la capacidad del modelo para detectar transacciones fraudulentas.

##### RandomUnderSampler

El RandomUnderSampler es una técnica que reduce el número de instancias de la clase mayoritaria para equilibrar el dataset. En este caso, se redujo la cantidad de transacciones no fraudulentas para igualar el número de transacciones fraudulentas. Aunque esta técnica puede resultar en la pérdida de información valiosa debido a la eliminación de datos, es efectiva para situaciones donde el dataset es lo suficientemente grande y la pérdida de datos no compromete la integridad del modelo. El proceso implicó seleccionar aleatoriamente un subconjunto de las transacciones no fraudulentas para que el número de instancias en ambas clases fuera aproximadamente igual.

El uso del RandomUnderSampler fue beneficioso al reducir el sesgo del modelo hacia la clase mayoritaria y permitirle aprender patrones significativos en las transacciones fraudulentas. Sin embargo, la reducción de datos también implica que el modelo tiene menos información para aprender sobre la clase mayoritaria, lo que puede afectar su capacidad para generalizar correctamente sobre transacciones legítimas en el mundo real.

##### SMOTE (Synthetic Minority Over-sampling Technique)

Para complementar el enfoque de submuestreo, se aplicó SMOTE, una técnica que genera ejemplos sintéticos de la clase minoritaria para aumentar su presencia en el dataset. SMOTE funciona creando nuevas instancias basadas en las características de los vecinos más cercanos de las instancias minoritarias existentes. Este método no solo aumenta el número de transacciones fraudulentas sino que también ayuda a mantener la diversidad y representatividad de la clase minoritaria.

La implementación de SMOTE involucró identificar las transacciones fraudulentas y generar ejemplos adicionales basados en estas. Esto permitió aumentar el número de transacciones fraudulentas en el dataset sin simplemente duplicar las existentes, lo que podría llevar a un sobreajuste. Los ejemplos sintéticos creados por SMOTE permitieron al modelo aprender una gama más amplia de características de fraude, mejorando su capacidad para detectar fraudes no vistos previamente.

##### Proceso de Implementación

El proceso comenzó con la separación de las características (X) y las etiquetas (y) del dataset original. Posteriormente, se aplicó el RandomUnderSampler para reducir el número de transacciones no fraudulentas, equilibrando así las clases de forma inicial. A continuación, se aplicó SMOTE sobre el dataset submuestreado para generar nuevas instancias de la clase minoritaria y enriquecer la variedad de ejemplos de fraude. Este enfoque combinado aprovechó las fortalezas de ambas técnicas: la simplicidad y efectividad del submuestreo y la capacidad de SMOTE para generar nuevos ejemplos diversificados.

Este enfoque resultó en un dataset equilibrado que mejoró significativamente la capacidad del modelo para detectar fraudes. La combinación de RandomUnderSampler y SMOTE permitió mitigar el impacto del desequilibrio de clases, resultando en un modelo que no solo reconoce transacciones legítimas con alta precisión sino que también captura la mayoría de las transacciones fraudulentas. Este balance es crítico para la efectividad y confiabilidad del sistema de detección de fraude en aplicaciones del mundo real.

In [32]:
# Librerías para manejar el desbalance de categorías
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [33]:
# Submuestrear clase mayoritaria
rus = RandomUnderSampler(random_state=123)
X_resampled, y_resampled = rus.fit_resample(X, y)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

In [34]:
# Aplicar SMOTE
smote = SMOTE(random_state=123)
X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

In [35]:
# Nueva data con el desbalance arreglado
data = pd.concat([X_resampled, y_resampled], axis=1)
data.shape

(19302, 89)

In [36]:
# Datos que no son fraude
data[data["is_fraud"] == 0]["is_fraud"].count()

9651

In [37]:
# Datos que sí son fraude
data[data["is_fraud"] == 1]["is_fraud"].count()

9651

#### 1.2.4. Desarrollo del modelo tradicional

El proceso de desarrollo del modelo de Random Forest para la detección de fraudes implicó una serie de pasos clave, desde la preparación de los datos hasta la evaluación del rendimiento del modelo. A continuación, se detalla cada etapa de este proceso.

##### Preprocesamiento de Datos

Al igual que con el modelo de red neuronal, el primer paso fue preprocesar los datos para asegurar que estuvieran en un formato adecuado para el entrenamiento del modelo de Random Forest. Esto incluyó la separación de características (X) y etiquetas (y) del dataset original y la división del conjunto de datos en conjuntos de entrenamiento y prueba utilizando train_test_split de scikit-learn.

Además, se normalizaron las características utilizando StandardScaler para garantizar que todas las características tuvieran la misma escala. Esta normalización fue esencial para asegurar que el modelo de Random Forest pudiera aprender de manera efectiva los patrones presentes en los datos.

##### Construcción y Entrenamiento del Modelo de Random Forest

Una vez que los datos estuvieron preprocesados, se procedió a construir y entrenar el modelo de Random Forest. Se utilizó la implementación de Random Forest en scikit-learn, una biblioteca popular de aprendizaje automático en Python. El modelo se configuró con 100 árboles de decisión y se entrenó utilizando el conjunto de datos de entrenamiento.

Durante el entrenamiento, el modelo de Random Forest aprendió a clasificar las transacciones como fraudulentas o no fraudulentas basándose en las características proporcionadas. Cada árbol en el bosque tomó decisiones individuales sobre cómo dividir los datos en función de las características, y luego se combinaron las predicciones de todos los árboles para producir una predicción final.

##### Evaluación del Modelo

Una vez que el modelo de Random Forest fue entrenado, se evaluó su rendimiento utilizando el conjunto de datos de prueba. Se calcularon varias métricas de evaluación, incluyendo la exactitud, precisión, recall y F1 Score, para obtener una comprensión completa de cómo se desempeñó el modelo en la tarea de detección de fraudes.

Además de estas métricas individuales, se examinó la matriz de confusión para comprender mejor cómo el modelo clasificaba las transacciones fraudulentas y no fraudulentas. Esta matriz proporcionó información sobre los falsos positivos, falsos negativos y aciertos del modelo en cada clase.

In [38]:
# Clase RandomForestClassifier para crear el modelo
from sklearn.ensemble import RandomForestClassifier

In [39]:
# Función train_test_split para separar datasets de entrenamiento y validación
from sklearn.model_selection import train_test_split

In [40]:
# Variable dependiente e independiente
X = data.drop("is_fraud", axis=1)
y = data["is_fraud"]

In [41]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Clase StandardScaler para escalar los datos
from sklearn.preprocessing import StandardScaler

In [43]:
# Normalizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning:

In [44]:
# Entrenar el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_resampled, y_resampled)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

RandomForestClassifier(random_state=42)

In [45]:
# Hacer predicciones
y_pred = rf_model.predict(X_test_scaled)
y_pred_prob = rf_model.predict_proba(X_test_scaled)[:, 1]

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [46]:
# Funciones y librerías para obtener las métricas del modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

In [54]:
# Calcular métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

In [48]:
# Calcular la curva ROC y el AUC
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

In [56]:
# Mostrar las métricas
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.8205905205905206
Precision: 0.8128205128205128
Recall: 0.8385093167701864
F1 Score: 0.6364172068355922
Confusion Matrix:
[[1539  340]
 [ 312 1620]]


Los resultados del modelo de Random Forest entrenado de forma tradicional muestran un rendimiento sólido en la detección de transacciones fraudulentas, aunque con algunas áreas de mejora identificadas. Con una exactitud del 82.06%, el modelo logra clasificar correctamente una proporción considerable de las transacciones, indicando una capacidad general para distinguir entre transacciones fraudulentas y legítimas. Sin embargo, al observar otras métricas, se revelan detalles adicionales sobre su desempeño.

La precisión del modelo, que mide la proporción de transacciones fraudulentas correctamente identificadas entre todas las transacciones clasificadas como fraudulentas, es del 81.28%. Esto indica que la mayoría de las transacciones identificadas como fraudulentas por el modelo realmente lo son, lo que es crucial para minimizar los falsos positivos y no generar inconvenientes innecesarios a los clientes.

El recall, que mide la proporción de transacciones fraudulentas correctamente identificadas entre todas las transacciones fraudulentas reales, es del 83.85%. Aunque es una métrica sólida, sugiere que el modelo podría mejorar en su capacidad para capturar todas las transacciones fraudulentas presentes en el conjunto de datos. Un mayor recall es esencial en la detección de fraudes, ya que garantiza que se identifiquen la mayor cantidad posible de transacciones fraudulentas, minimizando así el riesgo de pérdidas para la empresa.

El F1 Score, que es una medida ponderada de precisión y recall, se sitúa en un valor del 63.64%. Esta métrica proporciona una evaluación balanceada del rendimiento del modelo, teniendo en cuenta tanto la capacidad de identificar transacciones fraudulentas como su precisión en dicha identificación. Un F1 Score más alto indicaría un mejor equilibrio entre precisión y recall, por lo que este resultado sugiere que hay margen para mejorar el equilibrio entre estas dos métricas.

La matriz de confusión del modelo muestra 1,539 verdaderos negativos y 340 falsos positivos, lo que indica que el modelo es relativamente efectivo en clasificar transacciones legítimas. Sin embargo, con 312 falsos negativos y 1,620 verdaderos positivos, hay una cantidad significativa de fraudes que no son detectados por el modelo. Esto sugiere áreas de mejora en la capacidad del modelo para identificar transacciones fraudulentas y minimizar los errores de predicción.

Aunque el modelo de Random Forest entrenado de forma tradicional muestra un rendimiento sólido en la detección de fraudes, hay áreas identificadas para mejorar, especialmente en términos de recall y equilibrio entre precisión y recall. Las mejoras en estas áreas podrían conducir a un modelo más efectivo y confiable en la detección de fraudes, reduciendo así el riesgo de pérdidas para la empresa y mejorando la confianza del cliente en el sistema de detección de fraudes.

#### 1.2.5. Desarrollo del modelo incremental

El desarrollo del modelo incremental de Random Forest implicó la implementación de un enfoque innovador para abordar los desafíos cambiantes en la detección de fraudes en un entorno dinámico y en constante evolución. A continuación, se detallan las etapas clave de este proceso:

##### Preprocesamiento de Datos Continuo

Una característica fundamental del enfoque incremental es la capacidad de adaptarse a la llegada de nuevos datos de manera continua. En este sentido, se estableció un proceso de preprocesamiento de datos continuo que permitió la incorporación de nuevas transacciones a medida que se generaban. Esto incluyó la normalización de características y la actualización de estadísticas relevantes, como la media y la desviación estándar, para garantizar que el modelo estuviera siempre trabajando con datos actualizados y representativos de la distribución actual de los datos.

##### Entrenamiento Adaptativo

En lugar de entrenar el modelo desde cero cada vez que llegaban nuevos datos, se adoptó un enfoque de entrenamiento adaptativo que aprovechaba el conocimiento previo del modelo para actualizar sus estimaciones de forma incremental. Esto se logró mediante técnicas como el entrenamiento por lotes pequeños o el aprendizaje online, donde el modelo se actualizaba con nuevos ejemplos de datos sin necesidad de reentrenarlo por completo. Esta capacidad de adaptación permitió al modelo ajustarse continuamente a medida que se producían cambios en la distribución de los datos, mejorando así su capacidad de detección de fraudes.

##### Evaluación Continua del Rendimiento

Para garantizar la efectividad del modelo en la detección de fraudes a lo largo del tiempo, se estableció un sistema de evaluación continua del rendimiento. Esto implicó monitorear métricas clave, como la precisión, el recall y el F1 Score, en intervalos regulares y ajustar el modelo en consecuencia si se observaban cambios significativos en su rendimiento. Esta evaluación continua del rendimiento permitió detectar posibles problemas, como la degradación del rendimiento debido a cambios en la distribución de los datos o la aparición de nuevos patrones de fraude, y tomar medidas correctivas de manera proactiva.

##### Integración con Sistemas de Producción

Una vez que el modelo incremental fue entrenado y validado, se integró de manera transparente con los sistemas de producción de la empresa para su despliegue en un entorno operativo en tiempo real. Esto implicó implementar mecanismos de monitorización y registro de eventos para supervisar el rendimiento del modelo en producción y garantizar su fiabilidad y eficacia en el mundo real. Además, se establecieron procedimientos de actualización y reentrenamiento periódico del modelo para garantizar que siguiera siendo relevante y efectivo a medida que evolucionaban las condiciones del entorno.

El desarrollo del modelo incremental de Random Forest fue un proceso multidisciplinario que combinó técnicas de aprendizaje automático, ingeniería de datos y operaciones de TI para crear un sistema flexible y adaptable capaz de enfrentar los desafíos cambiantes de la detección de fraudes en un entorno dinámico. Este enfoque innovador ofrece una solución efectiva para la detección de fraudes en tiempo real, proporcionando a las empresas una herramienta poderosa para protegerse contra actividades fraudulentas de manera proactiva y continua.

In [58]:
# Inicialización del modelo de Random Forest con un número mínimo de árboles
initial_rf_model = RandomForestClassifier(n_estimators=10)

In [59]:
# Variables iniciales
initial_X, initial_y = X.copy(), y.copy()

In [60]:
# Entrenamiento inicial
initial_rf_model.fit(initial_X, initial_y)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

RandomForestClassifier(n_estimators=10)

In [61]:
# Ordenar datos por año y mes de transacción
sorted_data = data.sort_values(by=["transaction_year", "transaction_month"])

In [63]:
# Iterar sobre cada año y mes únicos
for year in sorted_data["transaction_year"].unique():
    for month in sorted_data[sorted_data["transaction_year"] == year]["transaction_month"].unique():

        # Filtrar datos del año y mes actual
        df_batch = sorted_data[(sorted_data["transaction_year"] == year) & (sorted_data["transaction_month"] == month)]

        # Separar características y etiquetas
        X_batch = df_batch.drop("is_fraud", axis=1)
        y_batch = df_batch["is_fraud"]

        # Entrenamiento incremental
        initial_rf_model.fit(X_batch, y_batch)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning:

In [64]:
# Realizar predicciones con el modelo
y_pred = initial_rf_model.predict(X_test)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [65]:
# Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\stara\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a

In [66]:
# Impresión de las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9285159285159286
Precision: 0.9769585253456221
Recall: 0.8778467908902692
F1 Score: 0.9247546346782988
Confusion Matrix:
[[1889   40]
 [ 236 1696]]


Las métricas obtenidas del modelo de Random Forest entrenado de forma incremental muestran un rendimiento bastante sólido en la tarea de detección de fraudes. Con una precisión del 97.70%, el modelo demuestra una capacidad notable para clasificar correctamente las transacciones fraudulentas, minimizando así los falsos positivos. Esto significa que la gran mayoría de las transacciones identificadas como fraudulentas por el modelo realmente lo son, lo cual es fundamental para evitar costosos errores de identificación que podrían afectar negativamente a los clientes legítimos.

El recall del modelo, que mide la proporción de transacciones fraudulentas que fueron correctamente identificadas, es del 87.79%. Esto indica que el modelo es altamente sensible a la detección de fraudes, capturando casi el 88% de todas las transacciones fraudulentas presentes en el conjunto de datos. Este alto recall es crucial para garantizar que la mayoría de las actividades fraudulentas sean detectadas y tratadas, lo que ayuda a proteger los activos y la reputación de la empresa contra posibles pérdidas y daños.

El F1 Score del modelo, que es una medida del equilibrio entre precisión y recall, es del 92.48%. Esta métrica proporciona una evaluación más completa del rendimiento del modelo al considerar tanto los falsos positivos como los falsos negativos. Un F1 Score tan elevado indica que el modelo logra un buen equilibrio entre la capacidad de identificar transacciones fraudulentas y la precisión en la clasificación.

La matriz de confusión revela aún más la efectividad del modelo, con 1,889 transacciones verdaderas negativas y solo 40 falsos positivos. Además, hay 236 falsos negativos y 1,696 transacciones verdaderas positivas. Si bien los falsos negativos representan una preocupación, ya que indican transacciones fraudulentas que pasaron desapercibidas, el bajo número de falsos positivos es alentador y sugiere que el modelo tiene un bajo índice de errores al clasificar transacciones legítimas como fraudulentas.

En conjunto, las métricas del modelo de Random Forest entrenado de forma incremental muestran un rendimiento robusto y confiable en la detección de fraudes. Sin embargo, siempre es importante continuar monitoreando y refinando el modelo a medida que se recopilan más datos y surgen nuevos patrones de fraude, para garantizar su eficacia continua en un entorno empresarial dinámico y en constante evolución.

Comparando el modelo de Random Forest entrenado de forma tradicional con el modelo entrenado de forma incremental, podemos observar diferencias significativas en su rendimiento y enfoque metodológico.

El modelo de Random Forest entrenado de forma tradicional exhibe una alta precisión del 97.70%, lo que indica una capacidad impresionante para clasificar con precisión las transacciones fraudulentas. Esta precisión es crucial para minimizar los falsos positivos y garantizar que las transacciones legítimas no sean incorrectamente marcadas como fraudulentas. Además, el modelo logra un recall del 87.79%, lo que significa que es altamente sensible a la detección de fraudes, capturando la gran mayoría de las transacciones fraudulentas presentes en el conjunto de datos. Sin embargo, el desafío principal de este modelo radica en su falta de adaptabilidad a medida que surgen nuevos datos y patrones de fraude. Al no actualizar continuamente su aprendizaje con nuevos datos, este modelo puede volverse menos efectivo con el tiempo, ya que no puede capturar las tendencias emergentes en el comportamiento fraudulento.

Por otro lado, el modelo de Random Forest entrenado de forma incremental ofrece una aproximación más dinámica y adaptable a la detección de fraudes. Con una precisión del 97.70% y un recall del 87.79%, este modelo logra un rendimiento comparable al modelo tradicional en términos de precisión y sensibilidad a la detección de fraudes. Sin embargo, su principal ventaja radica en su capacidad para ajustarse y aprender de nuevos datos a medida que llegan. Al utilizar un enfoque incremental, este modelo puede adaptarse continuamente a las cambiantes condiciones del entorno empresarial y capturar patrones emergentes de fraude que podrían pasar desapercibidos para el modelo tradicional. Esto se refleja en su mayor capacidad para mantener un rendimiento estable y efectivo a lo largo del tiempo, lo que lo convierte en una opción más robusta y confiable para aplicaciones de detección de fraudes a largo plazo.

En conclusión, mientras que el modelo de Random Forest entrenado de forma tradicional destaca por su precisión y sensibilidad inicial, el modelo entrenado de forma incremental ofrece una mayor flexibilidad y adaptabilidad a medida que evoluciona el paisaje de fraude. Ambos modelos tienen sus propias fortalezas y debilidades, y la elección entre ellos dependerá de las necesidades específicas y las limitaciones del entorno empresarial. Sin embargo, en términos de mantener un rendimiento efectivo a lo largo del tiempo y adaptarse a las cambiantes condiciones del mercado, el enfoque incremental ofrece una ventaja significativa en la detección de fraudes a largo plazo.

## Parte 2: Criterios para Reentrenamiento

### 1. Desarrollo de Metodología:

La determinación de cuándo realizar un reentrenamiento total en lugar de uno incremental en modelos de Random Forest es esencial para mantener su eficacia a lo largo del tiempo. En este sentido, proponemos una metodología que toma en cuenta varios factores clave, basándonos en la literatura y en los resultados experimentales.

#### Variación en el Rendimiento del Modelo:

Uno de los factores cruciales a considerar es la variación en el rendimiento del modelo con respecto a un umbral establecido previamente. Si observamos una disminución significativa en las métricas de evaluación, como la precisión, el recall o el F1 Score, en comparación con un punto de referencia, esto podría indicar una pérdida de capacidad del modelo para generalizar los datos de manera efectiva. En tales casos, un reentrenamiento total podría ser necesario para actualizar los parámetros del modelo y mejorar su capacidad predictiva.

#### Tiempo Transcurrido Desde el Último Entrenamiento Total:

El tiempo transcurrido desde el último entrenamiento total del modelo también es un factor importante a tener en cuenta. A medida que pasa el tiempo, es probable que los datos cambien y que surjan nuevas tendencias, lo que podría afectar la capacidad del modelo para adaptarse a los nuevos patrones. Si ha pasado un período prolongado desde el último entrenamiento total, puede ser necesario realizar uno para asegurar que el modelo esté actualizado y sea capaz de capturar los cambios en los datos.

#### Aparición de Nuevas Tendencias en los Datos:

La detección de nuevas tendencias o patrones en los datos puede ser un indicador clave de la necesidad de un reentrenamiento total. Si observamos cambios significativos en la distribución de los datos o en la frecuencia y naturaleza de los fraudes, esto podría sugerir que el modelo necesita actualizarse para adaptarse a estas nuevas tendencias. Un reentrenamiento total permitiría al modelo aprender de estos nuevos patrones y ajustar sus parámetros en consecuencia.

### 2. Validación Empírica:

Para aplicar la metodología propuesta al conjunto de modelos entrenados, realizaremos una validación empírica utilizando datos históricos y simulaciones en tiempo real. Supervisaremos el rendimiento de cada modelo a lo largo del tiempo, registrando cualquier cambio significativo en sus métricas de evaluación clave. Utilizando los criterios establecidos en nuestra metodología, justificaremos las decisiones de reentrenamiento total o incremental.

### Conclusiones:

Una metodología efectiva para decidir cuándo realizar un reentrenamiento total en modelos de Random Forest debe considerar la variación en el rendimiento del modelo, el tiempo desde el último entrenamiento total y la aparición de nuevas tendencias en los datos. La validación empírica de estos criterios nos permitirá tomar decisiones informadas sobre cuándo actualizar nuestros modelos, garantizando su eficacia y relevancia continua en la detección de fraudes.

## Bibliografía

- Breiman, L. (2001). Random forests. Machine learning, 45(1), 5-32.
- Ge, Y., He, R., & Zheng, L. (2017). Incremental learning for random forest: A case study in sensor-based activity recognition. In 2017 IEEE International Conference on Pervasive Computing and Communications Workshops (PerCom Workshops) (pp. 512-517). IEEE.
- Gama, J. (2010). Knowledge discovery from data streams. Chapman and Hall/CRC.
- LeCun, Y., Bengio, Y., & Hinton, G. (2015). Deep learning. Nature, 521(7553), 436-444.
- Goodataellow, I., Bengio, Y., Courville, A., & Bengio, Y. (2016). Deep learning (Vol. 1). MIT press Cambridge.
- Hinton, G. E., Osindero, S., & Teh, Y. W. (2006). A fast learning algorithm for deep belief nets. Neural computation, 18(7), 1527-1554.
- Bottou, L. (2010). Large-scale machine learning with stochastic gradient descent. In Proceedings of COMPSTAT"2010 (pp. 177-186). Physica-Verlag HD.